# CIS 194: Homework 10

In [1]:
import           Control.Applicative

import           Data.Char

-- A parser for a value of type a is a function which takes a String
-- represnting the input to be parsed, and succeeds or fails; if it
-- succeeds, it returns the parsed value along with the remainder of
-- the input.
newtype Parser a = Parser { runParser :: String -> Maybe (a, String) }

-- For example, 'satisfy' takes a predicate on Char, and constructs a
-- parser which succeeds only if it sees a Char that satisfies the
-- predicate (which it then returns).  If it encounters a Char that
-- does not satisfy the predicate (or an empty input), it fails.
satisfy :: (Char -> Bool) -> Parser Char
satisfy p = Parser f
  where
    f [] = Nothing    -- fail on the empty input
    f (x:xs)          -- check if x satisfies the predicate
                        -- if so, return x along with the remainder
                        -- of the input (that is, xs)
        | p x       = Just (x, xs)
        | otherwise = Nothing  -- otherwise, fail

-- Using satisfy, we can define the parser 'char c' which expects to
-- see exactly the character c, and fails otherwise.
char :: Char -> Parser Char
char c = satisfy (== c)

{- For example:

*Parser> runParser (satisfy isUpper) "ABC"
Just ('A',"BC")
*Parser> runParser (satisfy isUpper) "abc"
Nothing
*Parser> runParser (char 'x') "xyz"
Just ('x',"yz")

-}

-- For convenience, we've also provided a parser for positive
-- integers.
posInt :: Parser Integer
posInt = Parser f
  where
    f xs
      | null ns   = Nothing
      | otherwise = Just (read ns, rest)
      where (ns, rest) = span isDigit xs

## Exercise 1

First, you’ll need to implement a `Functor` instance for `Parser`.
*Hint*: You may find it useful to implement a function

```haskell
first :: (a -> b) -> (a,c) -> (b,c)
```

In [2]:
first :: (a -> b) -> (a, c) -> (b, c)
first f (a, c) = (f a, c)

In [3]:
instance Functor Parser where
  fmap :: (a -> b) -> Parser a -> Parser b
  fmap f (Parser runA) = Parser $ (first f <$>) . runA

## Exercise 2

Now implement an Applicative instance for `Parser`:

- `pure` a represents the parser which consumes no input and successfully returns a result of `a`.
- `p1 <*> p2` represents the parser which first runs `p1` (which will
consume some input and produce a function), then passes the
remaining input to `p2` (which consumes more input and produces
some value), then returns the result of applying the function to the
value. However, if either `p1` or `p2` fails then the whole thing should
also fail (put another way, `p1 <*> p2` only succeeds if both `p1` and
`p2` succeed).

In [4]:
instance Functor Parser where
  fmap :: (a -> b) -> Parser a -> Parser b
  fmap f (Parser runA) = Parser $ (first f <$>) . runA

In [5]:
instance Applicative Parser where
  pure a = Parser (\s -> Just (a, s))
  (Parser pf) <*> (Parser px) = Parser $ \s -> do
    (f, s') <- pf s
    (x, s'') <- px s'
    return (f x, s'')

So what is this good for? Recalling the Employee example from
class,

In [6]:
type Name = String
data Employee = Emp {name :: Name, phone :: String} deriving (Show)

we could now use the Applicative instance for `Parser` to make an
employee parser from name and phone parsers. That is, if
```haskell
parseName :: Parser Name
parsePhone :: Parser String

Emp <$> parseName <*> parsePhone :: Parser Employee
```

is a parser which first reads a name from the input, then a phone
number, and returns them combined into an `Employee` record. Of
course, this assumes that the name and phone number are right
next to each other in the input, with no intervening separators. We’ll
see later how to make parsers that can throw away extra stuff that
doesn’t directly correspond to information you want to parse.

In [7]:
runWordParser [] = Nothing
runWordParser s = Just (x, unwords xs)
  where
    (x : xs) = words s

parseName :: Parser Name
parseName = Parser runWordParser

parsePhone :: Parser String
parsePhone = Parser runWordParser

In [8]:
pe = Emp <$> parseName <*> parsePhone :: Parser Employee

In [9]:
runParser pe "alice 173"

Just (Emp {name = "alice", phone = "173"},"")

In [10]:
runParser pe "Bob 902 hs"

Just (Emp {name = "Bob", phone = "902"},"hs")

## Exercise 3

We can also test your `Applicative` instance using other simple
applications of functions to multiple parsers. You should implement
each of the following exercises using the `Applicative` interface to put
together simpler parsers into more complex ones. Do *not* implement
them using the low-level definition of a `Parser`! In other words, pretend that you do not have access to the `Parser` constructor or even
know how the Parser type is defined.

- Create a parser
```haskell
abParser :: Parser (Char, Char) 
```

which expects to see the characters `’a’` and `’b’` and returns them
as a pair. That is,
```
*AParser> runParser abParser "abcdef"
Just ((’a’,’b’),"cdef")
*AParser> runParser abParser "aebcdf"
Nothing
```

In [11]:
abParser :: Parser (Char, Char)
abParser = (,) <$> char 'a' <*> char 'b'

In [12]:
runParser abParser "abcdef"

Just (('a','b'),"cdef")

In [13]:
runParser abParser "aebcdf"

Nothing

- Create a parser `intPair` which reads two integer values separated
by a space and returns the integer values in a list. You should use
the provided `posInt` to parse the integer values

```
*Parser> runParser intPair "12 34"
Just ([12,34],"")
```

In [14]:
intPair :: Parser [Integer]
intPair = (\x _ y -> [x, y]) <$> posInt <*> char ' ' <*> posInt

In [15]:
runParser intPair "12 34"

Just ([12,34],"")

In [16]:
runParser posInt "12 34"

Just (12," 34")

## Exercise 4

`Applicative` by itself can be used to make parsers for simple, fixed
formats. But for any format involving *choice* ( *e.g.* “. . . after the colon
there can be a number **or** a word **or** parentheses. . . ”) `Applicative` is
not quite enough. To handle choice we turn to the `Alternative` class,
defined (essentially) as follows:

```haskell
class Applicative f => Alternative f where
    empty :: f a
    (<|>) :: f a -> f a -> f a
```

(<|>) is intended to represent choice: that is, f1 <|> f2 represents
a choice between f1 and f2. `empty` should be the identity element for
(<|>), and often represents *failure*.

Write an `Alternative` instance for `Parser`:

- `empty` represents the parser which always fails.
- p1 <|> p2 represents the parser which first tries running p1. If
p1 succeeds then p2 is ignored and the result of p1 is returned.
Otherwise, if p1 fails, then p2 is tried instead.

*Hint*: there is already an `Alternative` instance for `Maybe` which you
may find useful.

In [17]:
instance Alternative Parser where
  empty = Parser $ const Nothing
  (Parser r1) <|> (Parser r2) = Parser $ \s -> r1 s <|> r2 s

## Exercise 5

Implement a parser
```hasekll
intOrUppercase :: Parser () 
```
which parses either an integer value or an uppercase character, and
fails otherwise.

```
*Parser> runParser intOrUppercase "342abcd"
Just ((), "abcd")
*Parser> runParser intOrUppercase "XYZ"
Just ((), "YZ")
*Parser> runParser intOrUppercase "foo"
Nothing
```

In [18]:
intOrUppercase :: Parser ()
intOrUppercase = f posInt <|> f (satisfy isUpper)
  where
    f = (const () <$>)

In [19]:
runParser intOrUppercase "342abcd"

Just ((),"abcd")

In [20]:
runParser intOrUppercase "XYZ"

Just ((),"YZ")

In [21]:
runParser intOrUppercase "foo"

Nothing